### Alice in SEQUENCELAND 

##### Generating text based on Alice in wonderland with LSTM arquitecture 

In [1]:
import nltk
from nltk.corpus import gutenberg
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
alice = gutenberg.raw(fileids='carroll-alice.txt')

In [4]:
len(alice)

144395

In [5]:
alice[0:100]

"[Alice's Adventures in Wonderland by Lewis Carroll 1865]\n\nCHAPTER I. Down the Rabbit-Hole\n\nAlice was"

In [6]:
chars = sorted(list(set(alice)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 73


In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 30 # gives us an idea of the seed for the generation 
step = 2
sentences = []
next_chars = []


In [8]:
for i in range(0, len(alice) - maxlen, step):
    sentences.append(alice[i: i + maxlen])
    next_chars.append(alice[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 72183


In [9]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(alice) - maxlen - 1)
    for diversity in [0.2, 0.5, 0.7, 0.8]: #The higher diversity the randomness in the text 
        print('----- diversity:', diversity)

        generated = ''
        sentence = alice[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=32,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
72128/72183 [============================>.] - ETA: 0s - loss: 2.0788
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " wish to offend the Dormouse a"
 wish to offend the Dormouse and the was the was and the was a lare the was the was a louse the said the was a lare the said the was of the was a ringere the was a lore the soon the was the was a look the said the was the was a look the was and the was a lare the soon the was a look the was and the said the was a loor she went the was to herself whith the was a look the was the sas and the went the was a look the was a lore th
----- diversity: 0.5
----- Generating with seed: " wish to offend the Dormouse a"
 wish to offend the Dormouse and oned the made the took her sild thought house, and the madred the cannt the said it talk think the sinked the whild she the was she said the replith a vook in the was and hered the she wond of the sood the lask the was in the coor sone the goor to some t

'You're a mouse this, you know that it as too , they was nothing us so itces she the Mousely it head the bether was said.

'I don't his lews eat fald by the head the gretled
the sais to dee sharts might slobed a little its half she could,' said the would deal knew that a out it could his there ceat a time some dirw
----- diversity: 0.8
----- Generating with seed: " and bread-and-butter, and wen"
 and bread-and-butter, and went I done doesn't begind on any of ecome
coulddded so leary, as say tried stohe her had the plice it said as the Rlyste
usdening to had not out at ent behes.         oom him again the end to nengetmed ake the hooking but she contly such a little daic
it: 
'cears wain the
feepen bfieaw: she waited it so sure said to 'stad I begin but entored that soI should at the much at the word not to begry with 
72183/72183 [==============================] - 75s 1ms/step - loss: 1.5658
Epoch 5/60
72160/72183 [============================>.] - ETA: 0s - loss: 1.5292
----- Generati

things are "must that they had make a back a little of the tried to the turious as she said to the turious and the tapy the words a little the poor the same of the other thing as she said to the turious a come the turious a little the triean that she said to the time that she said to the sanch of the same of the same thing that she said the same of the room of the turious and was stright to the time the tapre t
----- diversity: 0.5
----- Generating with seed: "ou know you say
things are "mu"
ou know you say
things are "must said to the baby, she thought that is again was spoke the twead thought you over, sid the baby was or come of right and were the tuph she said a thing to them, 'or that's you was for say on this is of the baby moment of Alice had a great the shouse a piecos to distilly pastes that she said to the stard thair stall up and won't gat all she said the same that all of the door went on the head the 
----- diversity: 0.7
----- Generating with seed: "ou know you say
things

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


r was the table.

'It's the little was the table, and then the soon and the thing the tapped in a things to the table.

'It's no a thing to the table, and then she was the time she was to the it was to the twelf some took of the table.

'What was 
----- diversity: 0.5
----- Generating with seed: "a rat-hole: she knelt down and"
a rat-hole: she knelt down and she came she came in a nitited arems the soon of the farted!'

'It's the whole to sentone the other egited to see it was air,' said Alice, 'How were nothing of the
time another some said, and she was the leadted before this coming to the right the sort of for a bring to her as with her head the other as sitting to to the turtily to the other, she began to the table.

'It's not going to her it.

'
----- diversity: 0.7
----- Generating with seed: "a rat-hole: she knelt down and"
a rat-hole: she knelt down and with the soin a lobster
of the arce of the little under the but to grow it midht so say turus and in a or hat and looked
to se

...

'I don't disame too sme all inn't large and the I'll thing to find she heard, the Mock Turtle till began is anything, with the are their hardly found out and she went time in the little moral of her 
----- diversity: 0.8
----- Generating with seed: ", 'and that's the queerest thi"
, 'and that's the queerest thing,' said the great here,
sied again, and she was the litten spoke, and
bevere! Heit
yes. 'air, when they she celled the ecked to shar was with Alice shard was
like a conted end!' said the head low, prolded all quite among Add
to fance op of near,

'Well
grow all behothiny, and of her head her too like words a long
was would nett
glan can leary it again; and an she she round dlealed his would, say
72183/72183 [==============================] - 75s 1ms/step - loss: 1.5523
Epoch 44/60
72160/72183 [============================>.] - ETA: 0s - loss: 1.5745
----- Generating text after Epoch: 43
----- diversity: 0.2
----- Generating with seed: "Caterpillar called after her. "
Caterpilla

Alice was without the table.

'I should she before of herself of the arck a sort and she she it was to she she went on angers all then the are be to it to an then
this tine that there was a bit without them
a proce, withoulle began in the first ones to when began them the thing tone.

'Why see a tone, she it innothing to the other and withoull they.

Alice tuve and there they turn among the shorlre thing
----- diversity: 0.7
----- Generating with seed: "nd the fall was over.

Alice w"
nd the fall was over.

Alice was them, for
hand wonders thertly in a any what is tumbling thead.

'I or tume that, she turned it tombled not, withought get the pucunt; ?' and 'becimses it was of a trughad tother sholing was turgen bewal gordoimle. 
                                                                                                                                            at will woy, tindy
there
then that she a 
----- diversity: 0.8
----- Generating with seed: "nd the fall was over.

Alice 

Epoch 51/60
72160/72183 [============================>.] - ETA: 0s - loss: 1.6807
----- Generating text after Epoch: 50
----- diversity: 0.2
----- Generating with seed: "rised at her own courage. 'It'"
rised at her own courage. 'It's a little thing the looked to the first of the table the looking to she and she tonge and the looked and the looked to the look and the looked to the and the fan the end of the looked to the looked to the looked off the looked the looked to the table the house her said to her and the very heard and the looked to the table the looked to the looked to the looked to the fan the fan and the looking a
----- diversity: 0.5
----- Generating with seed: "rised at her own courage. 'It'"
rised at her own courage. 'It's her as she moment, that she could the looked some thing it would the first to shouls to she the time of a fan she great little the ey'
his head here!' And her began the looked the and the thing the thrnkilked in the next of the looked diffeut a little t

'I and she here:are the triwad, said-
'             I ymountes as in of of beag by Axce of she she he said-this beds all, there therx THedd swillle wonde in it sJtup me she for
of

'                                  And and she hid.se, stmes
----- diversity: 0.8
----- Generating with seed: "her shoulders
were nowhere to "
her shoulders
were nowhere to ceeras for it to be therx therx therx THet there whitdven
wase aaglire, and se you uularlion alit it,        And a oh crie cdoe fur maitr caster at che I've sionting at ale oumss all.

'We, dor said-
'I so therx they's flowe to fining oncees bedcaulung a very off, of to mape. 'Ohe Alice really, who baid!        T Osts eove els
stred of of bitd the an of there and 'empourns to meatrutieu
at in belo
72183/72183 [==============================] - 76s 1ms/step - loss: 2.6668
Epoch 55/60
72160/72183 [============================>.] - ETA: 0s - loss: 4.2634
----- Generating text after Epoch: 54
----- diversity: 0.2
----- Generating with seed: "

'  BeasatwUhin to cas[t the sewr*tiu reat on to oh chte KltesE''der Tre( wXn, and to an an tha onor are--'dis sert, yn egro' not -theer ltseorVW an an to and the sou ghe ssiiY, 'awat theJ anos she texksgn, saOed[oe 
----- diversity: 0.5
----- Generating with seed: "; 'I seem to see some meaning "
; 'I seem to see some meaning ta tolhe J   noeit hope
bitithieres FnorsBiuth, mouue, sa)
fou enr the teed-le ahdenwen1's    the me Toou ito the ur. 'the 'nooy to shddo laddeEaV line anwhV anere torahe ciditcu tf tha eissSpMbid theQZ the was un   Soume,on ihe ohla ar no oon a on *   iexn 'anst rut shy ide S Th so(sTwem lif
sSl'ss'theeElv  and was satornesge her, sZd avesehn stos loder to
t Th heV  fertherets a1-and ofrpchQen th
----- diversity: 0.7
----- Generating with seed: "; 'I seem to see some meaning "
; 'I seem to see some meaning sin ofdedeens lim encdUelly she Ah. 
sinE beand  dor Ththea' 
bout iliohiay wiln oneU*n 'she, AlicliA vfue, it elr
 I  I sonme smwhe th)ooe ;bat isr Te Tlowhue

In [ ]:
## Depending on the seed and the diversity the generated texts
## seem to have less or more meaning 